# ML Project

In [2]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 46.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
   

In [18]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
from os.path import exists

df_path = (
    "train.csv.zip"
    if exists("train.csv.zip")
    else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
)

if df_path.startswith("http"):
    response = requests.get(df_path)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        with z.open("train.csv") as f:
            df = pd.read_csv(f)
else:
    with zipfile.ZipFile(df_path, "r") as z:
        with z.open("train.csv") as f:
            df = pd.read_csv(f)

df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [19]:
df.head()
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [20]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [21]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType


enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)]
)

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
date,All valid,-
target,All valid,-




Running search request, search_id=37cf0e94-31e9-497c-8dd7-cc3889e1d119
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


44 relevant feature(s) found with the search keys: ['date']


item,11.7273,100.0000,"40, 30, 18",,,
store,7.9536,100.0000,"5, 10, 7",,,
f_events_date_week_sin1_847b5db1,3.5109,100.0000,"0.0, -0.4339, -0.9749",Upgini,Calendar data,Daily
f_autofe_roll_3d_min_1a2ad6d8bb,1.3821,100.0000,"0.3253, -0.2795, -0.3817","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,1.2849,100.0000,"1.0001, 1.0769, 1.0154",Upgini,Markets data,Daily
f_autofe_roll_3d_median_8e12f1e1e9,1.1626,100.0000,"0.309, -0.263, -0.3496","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_events_date_year_cos1_9014a856,1.0967,100.0000,"0.3253, -0.263, -0.3496",Upgini,Calendar data,Daily
f_events_date_week_cos3_7525fe31,0.8722,100.0000,"1.0, -0.2225, 0.6235",Upgini,Calendar data,Daily
f_autofe_roll_3d_q75_927f52faea,0.6021,100.0000,"33.0, 91.0, 31.5",Training dataset,AutoFE: features from Training dataset,
f_economic_date_cci_pca_8_54959a61,0.5847,100.0000,"-1.0818, -0.8969, -1.9917",Upgini,World economic indicators,Daily
f_autofe_roll_7d_median_e8edff6d9a,0.5017,100.0000,"83.0, 33.0, 100.0",Training dataset,AutoFE: features from Training dataset,


Upgini,Calendar data,5.8584,5
"Training dataset,Upgini","AutoFE: features from Training dataset,Markets data",3.1387,18
Training dataset,AutoFE: features from Training dataset,2.7830,10
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",2.5447,2
Upgini,Markets data,1.8015,5
Upgini,World economic indicators,0.8818,2


"Training dataset,Calendar data",f_autofe_roll_3d_min_1a2ad6d8bb,f_events_date_year_cos1_9014a856,roll_3d_min
"Training dataset,Calendar data",f_autofe_roll_3d_median_8e12f1e1e9,f_events_date_year_cos1_9014a856,roll_3d_median
Training dataset,f_autofe_roll_3d_q75_927f52faea,target,roll_3d_q75
Training dataset,f_autofe_roll_7d_median_e8edff6d9a,target,roll_7d_median
Training dataset,f_autofe_roll_3d_max_946bf6086b,target,roll_3d_max
"Training dataset,Markets data",f_autofe_roll_2d_norm_mean_f28c29d7cf,f_financial_date_vix_daydiff_02dc2726,roll_2d_norm_mean
Training dataset,f_autofe_roll_7d_max_f49b75b311,target,roll_7d_max
"Training dataset,Markets data",f_autofe_roll_2d_std_eb944d44c9,f_financial_date_vix_daydiff_02dc2726,roll_2d_std
"Training dataset,Markets data",f_autofe_lag_2d_82f0cadd32,f_financial_date_silver_7d_to_1y_0ccfe462,lag_2d
Training dataset,f_autofe_delta_7d_9b21a1d315,target,delta_7d
"Training dataset,Markets data",f_autofe_roll_2d_norm_mean_b7552e632c,f_financial_date_vix_gap_b64bd2b9,roll_2d_norm_mean


We detected 48 outliers in your sample.
Examples of outliers with maximum value of target:
33    205
17    196
12    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,9418,53.3352,0.324 ± 0.109,0.307 ± 0.099,0.0180,5.5%
Eval 1,3764,58.5994,0.278 ± 0.009,0.280 ± 0.035,-0.0020,-0.7%


In [24]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline MAPE,Enriched MAPE,"Uplift, abs","Uplift, %"
0,Train,9418,53.3352,0.288 ± 0.096,0.225 ± 0.085,0.063,21.8%
1,Eval 1,3764,58.5994,0.247 ± 0.008,0.204 ± 0.034,0.043,17.3%


In [31]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features =  enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

AttributeError: 'FeaturesEnricher' object has no attribute 'is_fitted'

In [34]:
model.fit(train_features, train_target)
enriched_pred = model.predict(test_features)
eval_metric(test_target.values, enriched_pred, "SMAPE")


[37.65141857448004]

In [36]:
model.fit(enriched_train_features, train_target)
enriched_pred = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_pred, "SMAPE")

CatBoostError: X must not be None